In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
gc.collect()
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages'

In [2]:
total_Dec17_Nov18_shoppers=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/RewardsShoppers_in_12_month_JL_2019-02-28.csv",
                                        usecols=["customer_id_hashed"])
total_Dec17_Nov18_shoppers=total_Dec17_Nov18_shoppers.drop_duplicates()
print("Total Shoppers Dec_2017 to Nov_2018: "+str(len(total_Dec17_Nov18_shoppers)))
del total_Dec17_Nov18_shoppers
gc.collect()

Total Shoppers Dec_2017 to Nov_2018: 16635546


14

In [3]:
selected_cols=['Customer_Link',
 'Event Time',
 'Advertiser ID',
 'Campaign ID',
 'Ad ID',
 'Rendering ID',
 'Creative Version',
 'Site ID (DCM)',
 'Placement ID',
 'Activity ID']

In [4]:
mapping_file_1=pd.read_table("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/CL_BigLots_dcm_account7252_activity_20181201_20181231_blDecTop5urls_20190129_135617_0000.tsv.gz",compression="gzip",sep="\t",usecols=selected_cols,dtype=str)
mapping_file_2=pd.read_table("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/CL_BigLots_dcm_account7252_activity_20181201_20181231_blNotLR_DecTop5urls_20190222_160759_0000.tsv.gz",compression="gzip",sep="\t",usecols=selected_cols,dtype=str)
mapping_file_3=pd.read_table("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/BL_top5URL_BigLots_LiveRamp_from_the_downloaded_folder_JL_2019-02-27.csv",dtype=str,usecols=selected_cols,sep=",")



In [5]:
mapping_file_all=mapping_file_1.append(mapping_file_2).append(mapping_file_3)
print(mapping_file_all.shape)
mapping_file_all=mapping_file_all.drop_duplicates()
print(mapping_file_all.drop_duplicates().shape)



(1131856, 10)
(1131855, 10)


In [6]:
mapping_file_all['Advertiser ID'].unique().tolist()

[nan, '8095847']

In [7]:
len(mapping_file_all['Placement ID'].unique().tolist())

1157

In [8]:
mapping_file_all['DateTime']=mapping_file_all['Event Time'].astype(int)/1000000-5*60*60
mapping_file_all['DateTime']=pd.to_datetime(mapping_file_all['DateTime'],unit='s')
mapping_file_all=mapping_file_all[mapping_file_all['Customer_Link']!="UNMATCHED"] # Filter out unmatched


In [9]:
all_rewards_mapping_Q4=pd.read_table("/home/jian/Projects/Big_Lots/Live_Ramp/Mapping_Files/2018_Q4/downloads_output/CL_BigLots_BL_mapping_file_2018Q4_JL_2019-02-01_20190207_025237_0000.psv.gz",
                                 sep="|",dtype=str,usecols=['Customer_Link','customer_id_hashed'])
all_rewards_mapping_Q4=all_rewards_mapping_Q4.drop_duplicates('Customer_Link')

In [10]:
len(all_rewards_mapping_Q4['customer_id_hashed'].unique().tolist())

23547622

In [11]:
mapping_file_all=pd.merge(mapping_file_all,all_rewards_mapping_Q4,on="Customer_Link",how="left")

In [12]:
mapping_file_all.shape

(757705, 12)

In [13]:
mapping_file_all.head(3)

,Customer_Link,Event Time,Advertiser ID,Campaign ID,Ad ID,Rendering ID,Creative Version,Site ID (DCM),Placement ID,Activity ID,DateTime,customer_id_hashed
0,XY1468I-HkyIq_no8POuQzTO4gsj19YCQyK9dOaRsfyeJhynE,1544663932493350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7945971,2018-12-12 20:18:52.493350000,NaN
1,XY14689w0AH6CfbBjcsehyBJpZtmgImzyf7c14bEk5syT0WZk,1544652622505450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7945971,2018-12-12 17:10:22.505449999,NaN
2,XY1468JEOdWHVMq58vSUZqgjICmL51gckvur1CNnvY3CxFz14,1544675498446709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7945980,2018-12-12 23:31:38.446709000,NaN


In [50]:
summary_by_site=mapping_file_all.groupby(['Activity ID','Campaign ID','Site ID (DCM)'])['Customer_Link'].apply(count_unique_counts).to_frame().reset_index()
site_name=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Site_Mapping_2018_Dec_only.csv",dtype=str)
summary_by_site=pd.merge(summary_by_site,site_name,on="Site ID (DCM)",how="left")

In [51]:
summary_by_site

,Activity ID,Campaign ID,Site ID (DCM),Customer_Link,Site (DCM)
0,7945971,20826194,3446099,5214,Criteo
1,7945971,20836313,1234535,44973,DART Search : Google
2,7945971,21219557,1284030,1,Pandora
3,7945971,21219557,2479413,332,The Trade Desk
4,7945971,21576806,1291657,1,Facebook
5,7945971,21629904,2479413,141,The Trade Desk
6,7945971,21653946,1291657,1,Facebook
7,7945971,21826343,1267626,1,Hulu.com
8,7945971,21826343,1271326,37,Tremor Video
9,7945971,21826343,1284030,28,Pandora


In [ ]:
mapping_file_all.groupby(,''])

In [14]:
del all_rewards_mapping_Q4
gc.collect()

114

In [15]:
# mapping_file_all[~pd.isnull(mapping_file_all['customer_id_hashed'])].shape
matched_activities_5_urls=mapping_file_all[~pd.isnull(mapping_file_all['customer_id_hashed'])]
matched_activities_5_urls.shape
# Matched rows

(89551, 12)

In [16]:
mapping_file_all[pd.isnull(mapping_file_all['customer_id_hashed'])].shape


(668154, 12)

In [17]:
def count_unique_id(x):
    return len(set(x))
matched_activities_5_urls['date']=matched_activities_5_urls['DateTime'].apply(lambda x: x.date())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
summary_unique_id_day_page=matched_activities_5_urls.groupby(["Activity ID",'date'])['customer_id_hashed'].apply(count_unique_id).to_frame().reset_index()
summary_unique_id_day_page.shape


(144, 3)

In [19]:
summary_unique_id_page=matched_activities_5_urls.groupby(["Activity ID"])['customer_id_hashed'].apply(count_unique_id).to_frame().reset_index()
summary_unique_id_page=summary_unique_id_page.sort_values('customer_id_hashed',ascending=False)
summary_unique_id_page

,Activity ID,customer_id_hashed
2,7945980,27143
0,7945971,10646
3,8001451,7988
1,7945974,3886
4,8001727,3707


In [20]:
matched_activities_5_urls=matched_activities_5_urls[['customer_id_hashed','Activity ID','DateTime','date']]

In [21]:
matched_activities_5_urls.shape

(89551, 4)

In [22]:
matched_activities_5_urls.drop_duplicates().shape

(89551, 4)

In [23]:
matched_activities_5_urls.head(2)

,customer_id_hashed,Activity ID,DateTime,date
3,a53300281a80925f670168111d449665c8143262e34dd3...,7945980,2018-12-12 10:07:13.726522,2018-12-12
4,fc5f499b7dc75219dccf9b678bec6c25d2968ad1deb3a6...,7945980,2018-12-12 09:26:36.967450,2018-12-12


In [24]:
def count_unique_counts(x):
    return len(set(x))

matched_exposed_ids_by_page=matched_activities_5_urls.groupby(['Activity ID'])['customer_id_hashed'].apply(count_unique_counts).to_frame().reset_index()

In [37]:
summary_unique_id_page

,Activity ID,customer_id_hashed
2,7945980,27143
0,7945971,10646
3,8001451,7988
1,7945974,3886
4,8001727,3707


In [38]:
matched_exposed_ids_by_page

,Activity ID,customer_id_hashed
0,7945971,10646
1,7945974,3886
2,7945980,27143
3,8001451,7988
4,8001727,3707


# Subclass Products Mapping

In [25]:
def clean_product_format(df):
    df['Class']=df['Class'].astype(int).astype(str)
    df['Subclass']=df['Subclass'].astype(int).astype(str)
    df=df.rename(columns={"Class":"class_code_id","Subclass":"subclass_id"})
    return df

In [26]:
Page_1_weekly_deals=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Tagged URL_Products_JL_20190228.xlsx",sheetname="Weekly Deals",dtype=str)
Page_1_weekly_deals['week_end_date']=Page_1_weekly_deals['week_end_date'].apply(lambda x: x[:10])
Page_1_weekly_deals=clean_product_format(Page_1_weekly_deals)

Page_1_weekly_deals_Week1=Page_1_weekly_deals[Page_1_weekly_deals['week_end_date']=="2018-12-08"]
Page_1_weekly_deals_Week2=Page_1_weekly_deals[Page_1_weekly_deals['week_end_date']=="2018-12-15"]
Page_1_weekly_deals_Week3=Page_1_weekly_deals[Page_1_weekly_deals['week_end_date']=="2018-12-22"]
Page_1_weekly_deals_Week4=Page_1_weekly_deals[Page_1_weekly_deals['week_end_date']=="2018-12-29"]

Page_2_Christmas_Shop=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Tagged URL_Products_JL_20190228.xlsx",sheetname="Christmas Shop",dtype=str)
del Page_2_Christmas_Shop['week_end_date']
Page_2_Christmas_Shop=clean_product_format(Page_2_Christmas_Shop)

Page_3_Christmas_Gifts=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Tagged URL_Products_JL_20190228.xlsx",sheetname="Christmas Gifts",dtype=str)
del Page_3_Christmas_Gifts['week_end_date']
Page_3_Christmas_Gifts=clean_product_format(Page_3_Christmas_Gifts)

Page_4_Toy_Shop=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Tagged URL_Products_JL_20190228.xlsx",sheetname="Toy Shop",dtype=str)
del Page_4_Toy_Shop['week_end_date']
Page_4_Toy_Shop=clean_product_format(Page_4_Toy_Shop)

Page_5_Holiday_Decorating=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/mapping_files/Tagged URL_Products_JL_20190228.xlsx",sheetname="Holiday Decorating",dtype=str)
del Page_5_Holiday_Decorating['week_end_date']
Page_5_Holiday_Decorating=clean_product_format(Page_5_Holiday_Decorating)


# Dec_Subclass_Sales

In [27]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [28]:
all_Q4_sales_subclass_files_2018Dec=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
all_Q4_sales_subclass_files_2018Dec=[x for x in all_Q4_sales_subclass_files_2018Dec if "aily" in x]
df_all_Q4_sales_subclass_files_2018Dec=pd.DataFrame({"file_path":all_Q4_sales_subclass_files_2018Dec})
df_all_Q4_sales_subclass_files_2018Dec.head(4)

df_all_Q4_sales_subclass_files_2018Dec['week_end_date']=df_all_Q4_sales_subclass_files_2018Dec['file_path'].apply(lambda x: x.split("MediaStorm_")[1][:10])
df_all_Q4_sales_subclass_files_2018Dec=df_all_Q4_sales_subclass_files_2018Dec[df_all_Q4_sales_subclass_files_2018Dec['week_end_date']>="2018-12-01"]
df_all_Q4_sales_subclass_files_2018Dec=df_all_Q4_sales_subclass_files_2018Dec[df_all_Q4_sales_subclass_files_2018Dec['week_end_date']!="2018-12-01"]
df_all_Q4_sales_subclass_files_2018Dec

,file_path,week_end_date
29,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-08
30,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-22
31,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-29
32,/home/jian/BigLots/2018_by_weeks/MediaStorm_20...,2018-12-15


In [29]:
df_all_Q4_sales_subclass_files_2018Dec['file_path'].tolist()

['/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-08/MediaStormDailySales20181211-115638-766.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-22/MediaStormDailySales20181225-112126-218.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-29/MediaStormDailySales20190101-113809-353.txt',
 '/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-15/MediaStormDailySales20181218-112627-166.txt']

In [30]:
samplerows=None
rewards_sales_Dec_4_weeks=pd.DataFrame()
for file in df_all_Q4_sales_subclass_files_2018Dec['file_path'].unique().tolist():
    df=pd.read_table(file, dtype=str, nrows=samplerows,sep="|")
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['subclass_transaction_units']=df['subclass_transaction_units'].astype(int)
    df=df[df['subclass_transaction_amt']>=0]
    print(datetime.datetime.now(),file)
    rewards_sales_Dec_4_weeks=rewards_sales_Dec_4_weeks.append(df)

2019-03-06 10:26:07.347051 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-08/MediaStormDailySales20181211-115638-766.txt
2019-03-06 10:26:56.402955 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-22/MediaStormDailySales20181225-112126-218.txt
2019-03-06 10:27:30.256076 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-29/MediaStormDailySales20190101-113809-353.txt
2019-03-06 10:28:16.856016 /home/jian/BigLots/2018_by_weeks/MediaStorm_2018-12-15/MediaStormDailySales20181218-112627-166.txt


### Page 2-5 4 weeks

In [31]:
# Do the page 2 - 5 first
'''
Page_Productdf_Dec_2_to_5=Page_2_Christmas_Shop.append(Page_3_Christmas_Gifts).append(Page_4_Toy_Shop).append(Page_5_Holiday_Decorating)
Page_Productdf_Dec_2_to_5=Page_Productdf_Dec_2_to_5.drop_duplicates()
Act_ID_2_to_5=Page_Productdf_Dec_2_to_5['Activity ID'].unique().tolist()


matched_activities_5_urls_2_to_5=matched_activities_5_urls[matched_activities_5_urls['Activity ID'].isin(Act_ID_2_to_5)]
print(matched_activities_5_urls_2_to_5.shape)
matched_activities_5_urls_2_to_5.head(2)

matched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5[['customer_id_hashed','Activity ID','date']].drop_duplicates(['customer_id_hashed','Activity ID'])
print(matched_activities_5_urls_2_to_5.shape)
matched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5.rename(columns={"date":"earliest_act_date"})
matched_activities_5_urls_2_to_5['earliest_act_date']=matched_activities_5_urls_2_to_5['earliest_act_date'].astype(str)
matched_activities_5_urls_2_to_5.head(2)
'''

'\nPage_Productdf_Dec_2_to_5=Page_2_Christmas_Shop.append(Page_3_Christmas_Gifts).append(Page_4_Toy_Shop).append(Page_5_Holiday_Decorating)\nPage_Productdf_Dec_2_to_5=Page_Productdf_Dec_2_to_5.drop_duplicates()\nAct_ID_2_to_5=Page_Productdf_Dec_2_to_5[\'Activity ID\'].unique().tolist()\n\n\nmatched_activities_5_urls_2_to_5=matched_activities_5_urls[matched_activities_5_urls[\'Activity ID\'].isin(Act_ID_2_to_5)]\nprint(matched_activities_5_urls_2_to_5.shape)\nmatched_activities_5_urls_2_to_5.head(2)\n\nmatched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5[[\'customer_id_hashed\',\'Activity ID\',\'date\']].drop_duplicates([\'customer_id_hashed\',\'Activity ID\'])\nprint(matched_activities_5_urls_2_to_5.shape)\nmatched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5.rename(columns={"date":"earliest_act_date"})\nmatched_activities_5_urls_2_to_5[\'earliest_act_date\']=matched_activities_5_urls_2_to_5[\'earliest_act_date\'].astype(str)\nmatched_activities_5_urls_2_to_5.

In [32]:
# Online & Shop Sales not differentiated 

def Dec_4weeks_page(Page_X_df,DCM_LR_Mapping_df,transaction_df):
    activity_id=Page_X_df['Activity ID'].unique().tolist()
    if len(activity_id)==1:
        activity_id=activity_id[0]
    else:
        print("Activity_id_Error: " + str(activity_id))
    DCM_LR_Mapping_df=DCM_LR_Mapping_df[DCM_LR_Mapping_df['Activity ID']==activity_id]
    DCM_LR_Mapping_df=DCM_LR_Mapping_df[['customer_id_hashed','date']].sort_values(['customer_id_hashed','date']).drop_duplicates('customer_id_hashed')
    DCM_LR_Mapping_df=DCM_LR_Mapping_df.rename(columns={"date":"earliest_date_on_site"})
    DCM_LR_Mapping_df['earliest_date_on_site']=DCM_LR_Mapping_df['earliest_date_on_site'].astype(str)
    
    page_X_df=Page_X_df[['Activity ID','class_code_id','subclass_id']]
    
    df_product_level=pd.merge(transaction_df,DCM_LR_Mapping_df,on=["customer_id_hashed"],how="left") # Add the LR catched ids
    
    df_product_level_all_products=pd.merge(df_product_level,page_X_df,on=["class_code_id",'subclass_id'],how="left")
    
    df_product_level_promoted_only=df_product_level_all_products[~pd.isnull(df_product_level_all_products['Activity ID'])] # Products on the site promoted only
    df_product_level_promoted_only['earliest_date_on_site']=df_product_level_promoted_only['earliest_date_on_site'].fillna("2019-01-31")
    df_product_level_promoted_only['Expose_Label']=np.where(df_product_level_promoted_only['earliest_date_on_site']<=df_product_level_promoted_only['transaction_dt'],"Exposed_Shopper","Not_Exposed_Shopper")
    
    id_with_expose_label=df_product_level_promoted_only[['customer_id_hashed','Expose_Label']].drop_duplicates()
    id_with_expose_label_date=df_product_level_promoted_only[['customer_id_hashed','transaction_dt','Expose_Label']].drop_duplicates()
    Exposed_NotBuy_df=DCM_LR_Mapping_df[~DCM_LR_Mapping_df['customer_id_hashed'].isin(id_with_expose_label[id_with_expose_label['Expose_Label']=="Exposed_Shopper"]['customer_id_hashed'].unique())]
    Exposed_NotBuy_count=len(Exposed_NotBuy_df['customer_id_hashed'].unique())
    ########### 1 Promoted Only for product level analysis
    ouotput_df_product_level_sales=df_product_level_promoted_only.groupby(['Expose_Label',"class_code_id",'subclass_id'])['subclass_transaction_units','subclass_transaction_amt'].sum().reset_index()
    
    ouotput_df_product_level_trans=df_product_level_promoted_only[['location_id','transaction_dt','transaction_id','customer_id_hashed','Expose_Label',"class_code_id",'subclass_id']].drop_duplicates()
    ouotput_df_product_level_trans=ouotput_df_product_level_trans.groupby(["class_code_id",'subclass_id','Expose_Label'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    ouotput_df_product_level_ids=df_product_level_promoted_only[['Expose_Label',"class_code_id",'subclass_id','customer_id_hashed']].drop_duplicates().groupby(["class_code_id",'subclass_id','Expose_Label'])['customer_id_hashed'].count().to_frame().reset_index()
    ouotput_df_product_level_promoted_only=pd.merge(ouotput_df_product_level_sales,ouotput_df_product_level_ids,on=['Expose_Label',"class_code_id",'subclass_id'],how="outer")
    
    ########### 2-1 Promoted Only
    ouotput_df_page_level_sales=df_product_level_promoted_only.groupby(['Expose_Label'])['subclass_transaction_units','subclass_transaction_amt'].sum().reset_index()
    ouotput_df_page_level_trans=df_product_level_promoted_only[['location_id','transaction_dt','transaction_id','customer_id_hashed','Expose_Label']].drop_duplicates()
    ouotput_df_page_level_trans=ouotput_df_page_level_trans.groupby(['Expose_Label'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    
    ouotput_df_page_level_ids=df_product_level_promoted_only[['Expose_Label','customer_id_hashed']].drop_duplicates().groupby(['Expose_Label'])['customer_id_hashed'].count().to_frame().reset_index()
    ouotput_df_page_level_promoted_only=pd.merge(ouotput_df_page_level_sales,ouotput_df_page_level_trans,on=['Expose_Label'],how="outer")
    ouotput_df_page_level_promoted_only=pd.merge(ouotput_df_page_level_promoted_only,ouotput_df_page_level_ids,on=['Expose_Label'],how="outer")
    ouotput_df_page_level_promoted_only['Products_Group']="Promoted_on_page"
    ########### 2-2
    df_product_level_all_products=pd.merge(df_product_level_all_products,id_with_expose_label_date,on=["customer_id_hashed","transaction_dt"],how="left")
    df_product_level_all_products=df_product_level_all_products[~pd.isnull(df_product_level_all_products['Expose_Label'])] # For the segments who purchanges at least 1 promoted products
    ouotput_df_page_level_sales=df_product_level_all_products.groupby(['Expose_Label'])['subclass_transaction_units','subclass_transaction_amt'].sum().reset_index()
    ouotput_df_page_level_trans=df_product_level_all_products[['location_id','transaction_dt','transaction_id','customer_id_hashed','Expose_Label']].drop_duplicates()
    ouotput_df_page_level_trans=ouotput_df_page_level_trans.groupby(['Expose_Label'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"transactions"})
    ouotput_df_page_level_ids=df_product_level_all_products[['Expose_Label','customer_id_hashed']].drop_duplicates().groupby(['Expose_Label'])['customer_id_hashed'].count().to_frame().reset_index()
    
    ouotput_df_page_level_all_products=pd.merge(ouotput_df_page_level_sales,ouotput_df_page_level_trans,on=['Expose_Label'],how="outer")
    ouotput_df_page_level_all_products=pd.merge(ouotput_df_page_level_all_products,ouotput_df_page_level_ids,on=['Expose_Label'],how="outer")
    ouotput_df_page_level_all_products['Products_Group']="All_Purchased"
    ###### 
    ouotput_df_page_level=ouotput_df_page_level_all_products.append(ouotput_df_page_level_promoted_only)
    
    ouotput_df_product_level_promoted_only['Page']=activity_id
    ouotput_df_page_level['Page']=activity_id
    
    print("Exposed_Not_Buy_promoted_products: "+str(Exposed_NotBuy_count))
    
    return ouotput_df_product_level_promoted_only,ouotput_df_page_level,Exposed_NotBuy_count

In [33]:
print(datetime.datetime.now())
Page_2_ouotput_df_product_level,Page_2_ouotput_df_page_level,Page_2_Exposed_NotBuy_count=Dec_4weeks_page(Page_X_df=Page_2_Christmas_Shop,
                                                                                                        DCM_LR_Mapping_df=matched_activities_5_urls,
                                                                                                        transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-06 10:28:21.608877


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Exposed_Not_Buy_promoted_products: 9439
2019-03-06 10:32:30.967967


In [34]:
print(datetime.datetime.now())
Page_3_ouotput_df_product_level,Page_3_ouotput_df_page_level,Page_3_Exposed_NotBuy_count=Dec_4weeks_page(Page_X_df=Page_3_Christmas_Gifts,
                                                                                                        DCM_LR_Mapping_df=matched_activities_5_urls,
                                                                                                        transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-06 10:32:30.975877


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Exposed_Not_Buy_promoted_products: 6895
2019-03-06 10:36:36.103226


In [35]:
print(datetime.datetime.now())
Page_4_ouotput_df_product_level,Page_4_ouotput_df_page_level,Page_4_Exposed_NotBuy_count=Dec_4weeks_page(Page_X_df=Page_4_Toy_Shop,
                                                                                                        DCM_LR_Mapping_df=matched_activities_5_urls,
                                                                                                        transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-06 10:36:36.112708


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Exposed_Not_Buy_promoted_products: 3490
2019-03-06 10:39:34.999610


In [36]:
print(datetime.datetime.now())
Page_5_ouotput_df_product_level,Page_5_ouotput_df_page_level,Page_5_Exposed_NotBuy_count=Dec_4weeks_page(Page_X_df=Page_5_Holiday_Decorating,
                                                                                                        DCM_LR_Mapping_df=matched_activities_5_urls,
                                                                                                        transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-06 10:39:35.007385


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Exposed_Not_Buy_promoted_products: 3198
2019-03-06 10:43:22.587364


In [39]:
writing_folder="/home/jian/Projects/Big_Lots/Analysis/2018_Q4/URL_Pages/output/"
try:
    os.stat(writing_folder)
except:
    os.mkdir(writing_folder)
    
writer=pd.ExcelWriter(writing_folder+"BL_Top_URLs_Page2_to_5_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

matched_exposed_ids_by_page.to_excel(writer,"Exposed_Matched_IDs",index=False)
Exposed_NotBuy_Df=pd.DataFrame({"Ppage":["Page_2_Exposed_NotBuy_count","Page_3_Exposed_NotBuy_count",
                                         "Page_4_Exposed_NotBuy_count","Page_5_Exposed_NotBuy_count"],
                                "IDs":[Page_2_Exposed_NotBuy_count,Page_3_Exposed_NotBuy_count,
                                       Page_4_Exposed_NotBuy_count,Page_5_Exposed_NotBuy_count]},index=[x for x in range(4)])
Exposed_NotBuy_Df.to_excel(writer,"Exposed_Not_Buy_IDs",index=False)

Page_2_ouotput_df_product_level.to_excel(writer,"Page_2_ouotput_df_product_level",index=False)
Page_2_ouotput_df_page_level.to_excel(writer,"Page_2_ouotput_df_page_level",index=False)

Page_3_ouotput_df_product_level.to_excel(writer,"Page_3_ouotput_df_product_level",index=False)
Page_3_ouotput_df_page_level.to_excel(writer,"Page_3_ouotput_df_page_level",index=False)

Page_4_ouotput_df_product_level.to_excel(writer,"Page_4_ouotput_df_product_level",index=False)
Page_4_ouotput_df_page_level.to_excel(writer,"Page_4_ouotput_df_page_level",index=False)

Page_5_ouotput_df_product_level.to_excel(writer,"Page_5_ouotput_df_product_level",index=False)
Page_5_ouotput_df_page_level.to_excel(writer,"Page_5_ouotput_df_page_level",index=False)

writer.save()

# Page_1 weekly deals 1 week only

In [163]:
def week_end_date(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [174]:
weekly_deals_activity_id=Page_1_weekly_deals['Activity ID'].unique().tolist()[0] #unique
matched_activities_Page1=matched_activities_5_urls[matched_activities_5_urls['Activity ID']==weekly_deals_activity_id]
matched_activities_Page1['week_end_date']=matched_activities_Page1['date'].apply(lambda x: week_end_date(x))
matched_activities_Page1.head(2)
print(matched_activities_Page1.shape)

(46530, 5)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [175]:
def Weekly_Deals_page(week_end_date_date,Page_Product_df,DCM_LR_Mapping_df,transaction_df):
    DCM_LR_Mapping_df=DCM_LR_Mapping_df[DCM_LR_Mapping_df['week_end_date']==week_end_date_date]
    
    Page_Product_df=Page_Product_df[Page_Product_df['week_end_date']==str(week_end_date_date)][['class_code_id','subclass_id','Activity ID']]
    
    for i_weekday in range(7):
        week_days_list=[str(week_end_date_date-datetime.timedelta(days=i_weekday)) for x in range(7)]
    
    transaction_df=transaction_df[transaction_df['transaction_dt'].isin(week_days_list)]
    transaction_df=pd.merge(transaction_df,Page_Product_df,on=['class_code_id','subclass_id'],how="left")
    transaction_df=transaction_df[~pd.isnull(transaction_df['Activity ID'])]
    
    DCM_LR_Mapping_df=DCM_LR_Mapping_df[['customer_id_hashed','date']].sort_values(['customer_id_hashed','date']).drop_duplicates('customer_id_hashed')
    DCM_LR_Mapping_df=DCM_LR_Mapping_df.rename(columns={"date":"earliest_date_on_site"})
    DCM_LR_Mapping_df['earliest_date_on_site']=DCM_LR_Mapping_df['earliest_date_on_site'].astype(str)
    df_product_level=pd.merge(transaction_df,DCM_LR_Mapping_df,on="customer_id_hashed",how="left")
    
    Exposed_NotBuy_df=DCM_LR_Mapping_df[~DCM_LR_Mapping_df['customer_id_hashed'].isin(df_product_level['customer_id_hashed'].unique())]
    Exposed_NotBuy_count=len(Exposed_NotBuy_df['customer_id_hashed'].unique())
    
    df_product_level['trans']=1
    df_product_level['earliest_date_on_site']=df_product_level['earliest_date_on_site'].fillna("2019-01-31")
    df_product_level['Expose_Label']=np.where(df_product_level['earliest_date_on_site']<=df_product_level['transaction_dt'],"Exposed_Buy","Not_Expose_Buy")
    
    ouotput_df_product_level_sales=df_product_level.groupby(['Expose_Label',"class_code_id",'subclass_id'])['subclass_transaction_units','subclass_transaction_amt','trans'].sum().reset_index()
    ouotput_df_product_level_ids=df_product_level[['Expose_Label',"class_code_id",'subclass_id','customer_id_hashed']].drop_duplicates().groupby(["class_code_id",'subclass_id','Expose_Label'])['customer_id_hashed'].count().to_frame().reset_index()
    ouotput_df_product_level=pd.merge(ouotput_df_product_level_sales,ouotput_df_product_level_ids,on=['Expose_Label',"class_code_id",'subclass_id'],how="outer")
    
    ouotput_df_page_level_sales=df_product_level.groupby(['Expose_Label'])['subclass_transaction_units','subclass_transaction_amt','trans'].sum().reset_index()
    ouotput_df_page_level_ids=df_product_level[['Expose_Label','customer_id_hashed']].drop_duplicates().groupby(['Expose_Label'])['customer_id_hashed'].count().to_frame().reset_index()
    ouotput_df_page_level=pd.merge(ouotput_df_page_level_sales,ouotput_df_page_level_ids,on=['Expose_Label'],how="outer")

    
    
    return ouotput_df_product_level,ouotput_df_page_level,Exposed_NotBuy_count

In [183]:
print(datetime.datetime.now())
WeeklyDeals_Prodcut_Level_Week_1,WeeklyDeals_Page_Level_Week_1,WeeklyDeals_Exposed_NotBuy_Count_Week_1=Weekly_Deals_page(week_end_date_date=datetime.date(2018,12,8),
                                                                                                                  Page_Product_df=Page_1_weekly_deals,
                                                                                                                  DCM_LR_Mapping_df=matched_activities_Page1,
                                                                                                                  transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())
WeeklyDeals_Exposed_NotBuy_Count_Week_1

2019-03-03 20:32:29.831248
2019-03-03 20:32:40.397567


6306

In [177]:
print(datetime.datetime.now())
WeeklyDeals_Prodcut_Level_Week_2,WeeklyDeals_Page_Level_Week_2,WeeklyDeals_Exposed_NotBuy_Count_Week_2=Weekly_Deals_page(week_end_date_date=datetime.date(2018,12,15),
                                                                                                                  Page_Product_df=Page_1_weekly_deals,
                                                                                                                  DCM_LR_Mapping_df=matched_activities_Page1,
                                                                                                                  transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-03 19:23:32.015353
2019-03-03 19:23:40.191014


In [178]:
print(datetime.datetime.now())
WeeklyDeals_Prodcut_Level_Week_3,WeeklyDeals_Page_Level_Week_3,WeeklyDeals_Exposed_NotBuy_Count_Week_3=Weekly_Deals_page(week_end_date_date=datetime.date(2018,12,22),
                                                                                                                  Page_Product_df=Page_1_weekly_deals,
                                                                                                                  DCM_LR_Mapping_df=matched_activities_Page1,
                                                                                                                  transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-03 19:23:40.200131
2019-03-03 19:23:56.188284


In [179]:
print(datetime.datetime.now())
WeeklyDeals_Prodcut_Level_Week_4,WeeklyDeals_Page_Level_Week_4,WeeklyDeals_Exposed_NotBuy_Count_Week_4=Weekly_Deals_page(week_end_date_date=datetime.date(2018,12,29),
                                                                                                                  Page_Product_df=Page_1_weekly_deals,
                                                                                                                  DCM_LR_Mapping_df=matched_activities_Page1,
                                                                                                                  transaction_df=rewards_sales_Dec_4_weeks)
print(datetime.datetime.now())

2019-03-03 19:23:56.195751
2019-03-03 19:24:09.399840


In [67]:
WeeklyDeals_Page_Level_Week_1

NameError: name 'WeeklyDeals_Page_Level_Week_1' is not defined

In [181]:
matched_activities_Page1.shape

(46530, 5)

In [97]:
rewards_sales_Dec_4_weeks['transaction_dt'].max()

'2018-12-29'

In [100]:
matched_activities_5_urls_2_to_5=matched_activities_5_urls[matched_activities_5_urls['Activity ID'].isin(Act_ID_2_to_5)]
print(matched_activities_5_urls_2_to_5.shape)
matched_activities_5_urls_2_to_5.head(2)

matched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5[['customer_id_hashed','Activity ID','date']].drop_duplicates(['customer_id_hashed','Activity ID'])
print(matched_activities_5_urls_2_to_5.shape)
matched_activities_5_urls_2_to_5=matched_activities_5_urls_2_to_5.rename(columns={"date":"earliest_act_date"})
matched_activities_5_urls_2_to_5['earliest_act_date']=matched_activities_5_urls_2_to_5['earliest_act_date'].astype(str)
matched_activities_5_urls_2_to_5.head(2)

(43021, 4)
(26227, 3)


,customer_id_hashed,Activity ID,earliest_act_date
41,d713ae551e5051ec4d5e45b2fdcb7d121123657b4ab856...,8001727,2018-12-12
97,b6651b06330498a071c6afbabc7e5d66e88b413c67d09e...,7945971,2018-12-12


In [74]:
matched_activities_5_urls.shape

(89551, 4)

In [75]:
matched_activities_5_urls.head(2)

,customer_id_hashed,Activity ID,DateTime,date
3,a53300281a80925f670168111d449665c8143262e34dd3...,7945980,2018-12-12 10:07:13.726522,2018-12-12
4,fc5f499b7dc75219dccf9b678bec6c25d2968ad1deb3a6...,7945980,2018-12-12 09:26:36.967450,2018-12-12


In [79]:
matched_activities_5_urls.shape

(89551, 4)

In [106]:
matched_activities_5_urls[(matched_activities_5_urls['date']>=datetime.date(2018,12,2)) & (matched_activities_5_urls['date']<=datetime.date(2018,12,30))].shape

(88046, 4)